<a href="https://colab.research.google.com/github/RajeshwariMankar/ML-Learning/blob/master/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install PyDrive**

In [8]:
! pip install PyDrive

    100% |████████████████████████████████| 993kB 19.9MB/s 
  Running setup.py bdist_wheel for PyDrive ... - \ done
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
from pydrive.drive import GoogleDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


**Get the list of files and their respective ids**

In [10]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: Digit_Dataset.zip, id: 16FMaIQBmrXZWVUGTxyuBYvvrxaqHHNIK
title: Untitled document, id: 1zGZWVU4PLX_-bWXP6coj9yf3Nh_O_nwX5uDk5goIL8o
title: Resume, id: 1NwjUrmHGh_Dt39M2-v7TXqQTqTPZycUj_sgXo9SvtVs
title: Untitled document, id: 1dCYN0NntH_8uHu1pNdl3awabXXIxY6Z4w2urNytxceA
title: Resume, id: 1I1YNt_IsnBE0M9wpxs-pJ9WrKHMHhGb0kULfBdnlzGk
title: Resume, id: 1Cs3Mn0KogPVdAG_IKGsrB8jw7as4mZp-Mc0XPYyIrko
title: Classifier.zip, id: 1I3xf9I0Snz_blVtE21L6pQbS-hVOPQJt
title: Untitled0.ipynb, id: 1X9tZ8r2QAi1-wzW-hEQOT4fkck6jxflr
title: Colab Notebooks, id: 1k5PSTWbW6iUE_lLvv8bCWZtpkwqLDXfc
title: WBC.zip, id: 1bqfWydpEeAVsmmQMDH_ij6ox6l-oFrxa
title: WBC.zip, id: 183TnHeAuJ81qjt-hhgohsRhTYRlCekQ1
title: WBC.zip, id: 1v7FjpEvExwBDU82UmF4MDhaTzg-nSdKq
title: TimeSheet, id: 1zkkGJN-k5PCNUsLlUHMekmhzaqwBtM3rWUBZGL6jSSk
title: companies_targeting_healthcare.doc, id: 0B97e4yRhem0ueVhNMzRJdGI5YzVDa0FXc2RDcEFLOGZCeWpJ
title: Cropped images.zip, id: 1COtHZp8haWllCEHKvFfu8c6O56R_MY2f
title: Cropped ima

**Get the file using its id**

In [0]:
f = drive.CreateFile({'id': '16FMaIQBmrXZWVUGTxyuBYvvrxaqHHNIK'})
f.GetContentFile('Digit_Dataset.zip')

**Unzip the file**

In [12]:
!unzip Digit_Dataset.zip -d /

Archive:  Digit_Dataset.zip
   creating: /Digit_Dataset/
   creating: /Digit_Dataset/training_set/
  inflating: /Digit_Dataset/training_set/train-images-idx3-ubyte  
  inflating: /Digit_Dataset/training_set/train-labels-idx1-ubyte  
   creating: /Digit_Dataset/test_set/
  inflating: /Digit_Dataset/test_set/t10k-images-idx3-ubyte  
  inflating: /Digit_Dataset/test_set/t10k-labels-idx1-ubyte  


**Import libraries**

In [0]:
import struct
import numpy as np
from sklearn import svm
from sklearn import neighbors,metrics
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
import matplotlib.pyplot as plt

**Read idx files and convert it to numpy array**

In [0]:
def read_idx(filename):
    with open(filename,'rb') as f:
        zero,data_type,dims = struct.unpack('>HBB',f.read(4))
        shape = tuple(struct.unpack('>I',f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read(), dtype = np.uint8).reshape(shape)

**Get the features and labels for training**

In [15]:
train = read_idx("/Digit_Dataset/training_set/train-images-idx3-ubyte")
print(train.shape)
#train = np.asarray(train)

train = train/255.0
train_features = train.reshape((len(train),-1))
print(train_features.shape)
train_labels = read_idx("/Digit_Dataset/training_set/train-labels-idx1-ubyte")
print(train_labels.shape)


test = read_idx("/Digit_Dataset/test_set/t10k-images-idx3-ubyte")
print(test.shape)
test = test/255.0
test_features = test.reshape((len(test), -1))
print(test_features.shape)
test_labels = read_idx("/Digit_Dataset/test_set/t10k-labels-idx1-ubyte")
print(test_labels.shape)

(60000, 28, 28)
(60000, 784)
(60000,)
(10000, 28, 28)
(10000, 784)
(10000,)


**Train the SVM Classifier**

In [16]:
# SVM with RBF Kernel
rbf_svc = svm.SVC(kernel='rbf', gamma=0.05, C=5)
rbf_svc.fit(train_features,train_labels)

SVC(C=5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.05, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

**Make Prediction**

In [0]:
prediction = rbf_svc.predict(test_features)

**Check the Performance of Classifier**

In [12]:
print(confusion_matrix(test_labels, prediction))  
print(classification_report(test_labels, prediction)) 

[[ 973    0    1    0    0    2    1    1    2    0]
 [   0 1127    3    1    0    1    0    1    2    0]
 [   4    0 1015    0    1    0    0    6    6    0]
 [   0    0    2  995    0    3    0    6    4    0]
 [   0    0    3    0  966    0    4    0    2    7]
 [   2    0    0    5    1  878    2    1    2    1]
 [   4    2    0    0    2    3  946    0    1    0]
 [   0    3   10    1    1    0    0 1004    2    7]
 [   1    0    1    4    1    2    0    2  960    3]
 [   3    3    2    6    9    2    0    5    6  973]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.98      0.98      0.98      1032
           3       0.98      0.99      0.98      1010
           4       0.98      0.98      0.98       982
           5       0.99      0.98      0.98       892
           6       0.99      0.99      0.99       958
           7       0.98      0.98   

**Check the train and test accuracy**

In [15]:
print(accuracy_score(prediction, test_labels))

0.9837
